In [1]:
import json
import os
from datasets import Dataset, Audio

audio = Audio(sampling_rate = 16000)

In [2]:
with open('emotion-questions.json') as fopen:
    questions = json.load(fopen)
    
questions = [q for q in questions if len(q) > 10]
len(questions)

85

In [3]:
with open('EmoV_DB.json') as fopen:
    data = json.load(fopen)

In [4]:
len(data)

6893

In [5]:
import random

gathered = []
for i in range(len(data)):
    filename = os.path.join('synthetic-EmoV_DB', f'{i}.json')
    if not os.path.exists(filename):
        continue
    
    with open(filename) as fopen:
        p = json.load(fopen)['response']
        
    gathered.append({
        'question': random.choice(questions),
        'answer': p,
        'metadata': json.dumps(data[i]),
        'audio_filename': data[i]['filename'],
        'source': 'CLAPv2/EmoV_DB'
    })
len(gathered)

6893

In [6]:
dataset = Dataset.from_list(gathered)

In [8]:
dataset[1]

{'question': 'What is the emotional character of this audio?',
 'answer': 'The tone of the audio suggests a composed and focused individual, carrying out their tasks with a sense of calm determination. The neutral emotion reflects a state of mindfulness and steady concentration without any apparent agitation or excitement.',
 'metadata': '{"filename": "EmoV_DB_audio/EmoV_DB_data_train_train-home_knoriy_fsx_processed_datasets_EmoV_db_EmoV_DB_tars_train_375-home_knoriy_fsx_processed_datasets_EmoV_db_EmoV_DB_tars_train_325-batch147_1.wav", "text": "He made sure that the magazine was loaded, and resumed his paddling.", "emotion": "Neutral"}',
 'audio_filename': 'EmoV_DB_audio/EmoV_DB_data_train_train-home_knoriy_fsx_processed_datasets_EmoV_db_EmoV_DB_tars_train_375-home_knoriy_fsx_processed_datasets_EmoV_db_EmoV_DB_tars_train_325-batch147_1.wav',
 'source': 'CLAPv2/EmoV_DB'}

In [9]:
dataset = dataset.cast_column("audio_filename", audio)
dataset

Dataset({
    features: ['question', 'answer', 'metadata', 'audio_filename', 'source'],
    num_rows: 6893
})

In [11]:
dataset.push_to_hub('mesolitica/Classification-Speech-Instructions', split = 'emotion_part2')